In [64]:
import pandas as pd

# Đọc dữ liệu từ file CSV hoặc database
user_df = pd.read_csv('user_cleaned.csv')
course_df = pd.read_csv('course_cleaned.csv')
user_video_df = pd.read_csv('user_video_cleaned.csv')

In [92]:
merged_df = user_df.merge(user_video_df, on='user_id', how='inner')
merged_df

,user_id,name,course_order,seq
0,U_112,姚保峰,"[696994, 735338, 696956, 677020, 697027, 78857...","[{'video_id': 'V_1395633', 'segment': [{'start..."
1,U_150,韩明瑞,"[682155, 697374]","[{'video_id': 'V_6224262', 'segment': [{'start..."
2,U_172,刘文博,"[948410, 936990, 1448497, 1471241, 697791, 676...","[{'video_id': 'V_6432959', 'segment': [{'start..."
3,U_189,张强,"[927961, 697334, 817148, 682155, 746971, 94325...","[{'video_id': 'V_6334508', 'segment': [{'start..."
4,U_197,李超,"[697791, 676937, 1794464, 808526, 948415, 6941...","[{'video_id': 'V_1358540', 'segment': [{'start..."
...,...,...,...,...
112396,U_26862793,NaN,"[697791, 676932]","[{'video_id': 'V_1358540', 'segment': [{'start..."
112397,U_26863000,发发,"[676932, 697068, 696767, 780629, 697791, 69699...","[{'video_id': 'V_1395633', 'segment': [{'start..."
112398,U_26863054,杨智婷,"[735031, 845950, 770784, 735149, 676662]","[{'video_id': 'V_1410123', 'segment': [{'start..."
112399,U_26863059,ZhangQJ,"[696892, 682350, 697270, 697423, 697155, 69703...","[{'video_id': 'V_6185387', 'segment': [{'start..."


In [93]:
def calculate_features(row):
    seq = row["seq"]
    
    # Kiểm tra xem seq có phải là danh sách không
    if isinstance(seq, str):
        seq = eval(seq)  # Chuyển chuỗi thành danh sách nếu seq là chuỗi (nếu seq được lưu dưới dạng chuỗi JSON)
    
    total_videos_watched = len(seq)  # Tổng số video đã xem
    total_watch_time = 0  # Tổng thời gian xem
    # total_speed = 0  # Tổng tốc độ
    # total_segments = 0  # Tổng số segment

    # Duyệt qua từng video trong seq
    for video in seq:
        segments = video.get("segment", [])  # Lấy segment nếu có, nếu không trả về danh sách rỗng
        for segment in segments:
            total_watch_time += segment.get("end_point", 0) - segment.get("start_point", 0)
            # total_speed += segment.get("speed", 0)
            # total_segments += 1

    # Tính tốc độ trung bình
    # avg_speed = total_speed / total_segments if total_segments > 0 else 0

    return pd.Series({
        "total_videos_watched": total_videos_watched,
        "total_watch_time": total_watch_time,
        # "avg_speed": avg_speed,
    })

# Áp dụng hàm tính toán cho từng hàng
features = merged_df.apply(calculate_features, axis=1)

# Gộp kết quả vào DataFrame
result_df = pd.concat([merged_df, features], axis=1)

# Xem kết quả
result_df

,user_id,name,course_order,seq,total_videos_watched,total_watch_time
0,U_112,姚保峰,"[696994, 735338, 696956, 677020, 697027, 78857...","[{'video_id': 'V_1395633', 'segment': [{'start...",6.0,584.200
1,U_150,韩明瑞,"[682155, 697374]","[{'video_id': 'V_6224262', 'segment': [{'start...",1.0,220.000
2,U_172,刘文博,"[948410, 936990, 1448497, 1471241, 697791, 676...","[{'video_id': 'V_6432959', 'segment': [{'start...",1.0,4.700
3,U_189,张强,"[927961, 697334, 817148, 682155, 746971, 94325...","[{'video_id': 'V_6334508', 'segment': [{'start...",20.0,6058.000
4,U_197,李超,"[697791, 676937, 1794464, 808526, 948415, 6941...","[{'video_id': 'V_1358540', 'segment': [{'start...",55.0,11304.248
...,...,...,...,...,...,...
112396,U_26862793,NaN,"[697791, 676932]","[{'video_id': 'V_1358540', 'segment': [{'start...",1.0,60.100
112397,U_26863000,发发,"[676932, 697068, 696767, 780629, 697791, 69699...","[{'video_id': 'V_1395633', 'segment': [{'start...",3.0,289.000
112398,U_26863054,杨智婷,"[735031, 845950, 770784, 735149, 676662]","[{'video_id': 'V_1410123', 'segment': [{'start...",3.0,665.960
112399,U_26863059,ZhangQJ,"[696892, 682350, 697270, 697423, 697155, 69703...","[{'video_id': 'V_6185387', 'segment': [{'start...",11.0,3915.000


In [94]:
# Xử lý course_order để đảm bảo tất cả là danh sách
def normalize_course_order(order):
    if isinstance(order, str):  # Nếu là chuỗi JSON, chuyển sang danh sách
        import ast
        return ast.literal_eval(order)
    elif isinstance(order, list):  # Nếu đã là danh sách
        return order
    else:  # Nếu giá trị khác, trả về danh sách rỗng
        return []

result_df['course_order'] = result_df['course_order'].apply(normalize_course_order)

# Hàm tính tổng số video của các khóa học trong course_order
def calculate_total_videos(course_order):
    total_videos = course_df.loc[course_df['id'].isin(course_order), 'total_videos'].sum()
    return total_videos

# Áp dụng hàm để tính tổng số video cần học
result_df['total_videos_required'] = result_df['course_order'].apply(calculate_total_videos)

# Tính tỉ lệ xem của người dùng
# result_df['watch_ratio'] = result_df['total_videos_watched'] / result_df['total_videos_required']
result_df

,user_id,name,course_order,seq,total_videos_watched,total_watch_time,total_videos_required
0,U_112,姚保峰,"[696994, 735338, 696956, 677020, 697027, 78857...","[{'video_id': 'V_1395633', 'segment': [{'start...",6.0,584.200,3505
1,U_150,韩明瑞,"[682155, 697374]","[{'video_id': 'V_6224262', 'segment': [{'start...",1.0,220.000,178
2,U_172,刘文博,"[948410, 936990, 1448497, 1471241, 697791, 676...","[{'video_id': 'V_6432959', 'segment': [{'start...",1.0,4.700,1177
3,U_189,张强,"[927961, 697334, 817148, 682155, 746971, 94325...","[{'video_id': 'V_6334508', 'segment': [{'start...",20.0,6058.000,1546
4,U_197,李超,"[697791, 676937, 1794464, 808526, 948415, 6941...","[{'video_id': 'V_1358540', 'segment': [{'start...",55.0,11304.248,394
...,...,...,...,...,...,...,...
112396,U_26862793,NaN,"[697791, 676932]","[{'video_id': 'V_1358540', 'segment': [{'start...",1.0,60.100,171
112397,U_26863000,发发,"[676932, 697068, 696767, 780629, 697791, 69699...","[{'video_id': 'V_1395633', 'segment': [{'start...",3.0,289.000,2250
112398,U_26863054,杨智婷,"[735031, 845950, 770784, 735149, 676662]","[{'video_id': 'V_1410123', 'segment': [{'start...",3.0,665.960,675
112399,U_26863059,ZhangQJ,"[696892, 682350, 697270, 697423, 697155, 69703...","[{'video_id': 'V_6185387', 'segment': [{'start...",11.0,3915.000,796


In [95]:
result_df = result_df[result_df['total_videos_required'] != 0]
result_df

,user_id,name,course_order,seq,total_videos_watched,total_watch_time,total_videos_required
0,U_112,姚保峰,"[696994, 735338, 696956, 677020, 697027, 78857...","[{'video_id': 'V_1395633', 'segment': [{'start...",6.0,584.200,3505
1,U_150,韩明瑞,"[682155, 697374]","[{'video_id': 'V_6224262', 'segment': [{'start...",1.0,220.000,178
2,U_172,刘文博,"[948410, 936990, 1448497, 1471241, 697791, 676...","[{'video_id': 'V_6432959', 'segment': [{'start...",1.0,4.700,1177
3,U_189,张强,"[927961, 697334, 817148, 682155, 746971, 94325...","[{'video_id': 'V_6334508', 'segment': [{'start...",20.0,6058.000,1546
4,U_197,李超,"[697791, 676937, 1794464, 808526, 948415, 6941...","[{'video_id': 'V_1358540', 'segment': [{'start...",55.0,11304.248,394
...,...,...,...,...,...,...,...
112396,U_26862793,NaN,"[697791, 676932]","[{'video_id': 'V_1358540', 'segment': [{'start...",1.0,60.100,171
112397,U_26863000,发发,"[676932, 697068, 696767, 780629, 697791, 69699...","[{'video_id': 'V_1395633', 'segment': [{'start...",3.0,289.000,2250
112398,U_26863054,杨智婷,"[735031, 845950, 770784, 735149, 676662]","[{'video_id': 'V_1410123', 'segment': [{'start...",3.0,665.960,675
112399,U_26863059,ZhangQJ,"[696892, 682350, 697270, 697423, 697155, 69703...","[{'video_id': 'V_6185387', 'segment': [{'start...",11.0,3915.000,796


In [96]:
result_df.to_csv('final_data.csv', index=False, encoding='utf-8')